In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

In [3]:
sys.path

['/Users/cuongdm8499/Me/git-cuongpiger/llm-evaluation',
 '/Users/cuongdm8499/Applications/PyCharm Professional Edition.app/Contents/plugins/python-ce/helpers/pydev',
 '/Users/cuongdm8499/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/opt/anaconda3/envs/llm-evaluation/lib/python310.zip',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/lib-dynload',
 '',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/setuptools/_vendor']

## Prepare the documents

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from collections import defaultdict
from langchain.schema import Document


In [7]:
directory_path = "./../data/vks/pdf/vi"  # Change this to your directory
md_files = []

for root, _, files in os.walk(directory_path):
    for file in files:
        if file.endswith(".pdf"):
            md_files.append(os.path.join(root, file))

print(md_files)

['./../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', './../data/vks/pdf/vi/02-nlb.pdf', './../data/vks/pdf/vi/04-thong-bao-cap-nhat-release-note.pdf', './../data/vks/pdf/vi/01-vks-la-gi.pdf']


In [13]:
docs = []

for file in md_files:
    loader = PyPDFLoader(file)

    data = loader.load()
    docs.extend(data)

docs

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-18T07:22:18+00:00', 'title': '02-mo-hinh-hoat-dong', 'moddate': '2025-02-18T07:22:18+00:00', 'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. T

In [16]:
# Group documents by source
merged_docs = defaultdict(str)
for doc in docs:
    merged_docs[doc.metadata["source"]] += (
            doc.page_content + "\n"
    )  # Concatenating with newline

In [19]:
new_docs = []

for source, doc in merged_docs.items():
    new_doc = Document(page_content=doc, metadata={"source": source})

    new_docs.append(new_doc)

In [20]:
new_docs

[Document(metadata={'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. Tuy nhiên bạn hoàn toàn có thể hạn chế truy cập đến\nControlPlane thông qua tính năng Whitelist IP. Một điều lưu ý là bạn cần đảm bảo không\nhạn chế truy cập từ các IP của các node trong nodegroup.\nPublic Cluster với Private Node Group: Các node trong nodegroup không có địa chỉ IP\nPublic riêng, không thể kết nối trực tiếp đế

In [21]:
len(new_docs)

4

## Chunking and save into vector-store database

In [30]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

In [32]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)
vector_store = InMemoryVectorStore(embeddings_model)

In [29]:
chunk_size = 7000
chunk_overlap = 6800
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

In [33]:
splits = text_splitter.split_documents(new_docs)

In [34]:
vector_store.add_documents(splits)  # save documents into vectorstore

['9923f519-c220-42bf-8ce5-678e6022b4c3',
 '9cb9a8f9-bfff-4767-978e-6214ec8ce1d8',
 '49eeba48-b59f-49ee-8f68-81ff741bd731',
 '108a1a70-a082-4b04-a0bd-9022259a2364',
 'ad6c50fe-7736-4572-be50-c9ff4ac5c564',
 '0a95f80c-3e77-49a8-a00d-70530d5262aa',
 '399d4572-446f-461a-ba23-9b8bb409f634',
 'fb120de8-fb9c-4ef2-a410-72396137663c',
 '560852d0-6c5d-4dc7-b0a9-839a447fdf80',
 'aff4d407-dccd-47fd-92c6-6b93d9a973fa',
 '738b377d-ad19-4811-82c4-db487e919a8d',
 'da30770b-6bcf-4977-8802-8cb2ce64a508',
 '1e552bd4-3090-48b0-8dd0-d883ff3c15e9',
 'f08b4fd8-5733-4d03-a6ce-61aae1e9861d',
 '59b3bc46-3c4c-4f7e-a444-6cc39e6519b7',
 'b9262b84-9af3-477e-a936-8758a78d5069',
 '4a7118c6-9b3b-41c8-8b29-bdfc4b22aae4',
 'c2e6b923-3406-4534-872d-80b359e78c1f',
 'dd87fec5-4fce-4653-a19b-1e1e19cc562a',
 '63b08a87-6fef-47cc-9920-4b86fbd1a600',
 'fc890b93-9cd4-4885-a896-2873e2cf01a9',
 '7921207d-85d9-4bfa-86bb-709760cc9fbd',
 '27d9577d-4572-4e6f-8db7-32e3b4c577c0',
 '6ecf6958-22f8-44d3-ba92-795678753625',
 'a9ce21c9-9e77-

In [35]:
retriever = vector_store.as_retriever()

In [38]:
query = "VKS là gì?"
for i, doc in enumerate(retriever.invoke(query)):
    print(f"Document {i}:\n{doc}", end="\n\n")

Document 0:
page_content='1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Ku

## Configure some environment variables

In [39]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

## Setup RAG chain

In [42]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [43]:
llm = ChatVertexAI(model_name="gemini-1.5-flash", temperature=0)


In [45]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
# Setup RAG pipeline
rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)


## Preparing the Evaluation Data

In [84]:
from ragas import EvaluationDataset, SingleTurnSample

In [91]:
questions = ["VKS có những tính năng gì nổi bật?",
             "VKS là gì?",
             "So sánh public và private cluster trong VKS",
             ]

ground_truths = [
    """
    Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau:

    Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.
    Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.
    Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.
    Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng.
    Scaling & Healing Automatically: VKS tự động mở rộng Node group khi cần thiết và tự động sửa lỗi khi node gặp vấn đề, giúp bạn tiết kiệm thời gian và công sức quản lý.
    Giảm chi phí và nâng cao độ tin cậy: VKS triển khai Control Plane của Kubernetes ở chế độ sẵn sàng cao và hoàn toàn miễn phí.
    Tích hợp Blockstore Native (Container Storage Interface - CSI): VKS cho phép bạn quản lý Blockstore thông qua YAML của Kubernetes, cung cấp lưu trữ bền vững cho container.
    Tích hợp Load Balancer (Network Load Balancer, Application Load Balancer): VKS cung cấp khả năng quản lý NLB/ALB thông qua YAML của Kubernetes, giúp bạn dễ dàng expose Service trong Kubernetes ra bên ngoài.
    Nâng cao bảo mật: VKS cho phép bạn tạo Private Node Group với chỉ Private IP và kiểm soát quyền truy cập vào cluster.
    Ngoài ra, VKS còn có các ưu điểm sau:

    Dễ sử dụng: VKS cung cấp giao diện đơn giản và dễ sử dụng.
    Chi phí hợp lý: VKS cung cấp mức giá cạnh tranh cho các dịch vụ của mình.
    Bạn có muốn tìm hiểu thêm về tính năng nào cụ thể của VKS không?
    """,
    """
    VKS là viết tắt của VNGCloud Kubernetes Service.

    Nó là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud.

    VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách dễ dàng và hiệu quả. Nó giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

    Nói cách khác, VKS là một giải pháp giúp bạn dễ dàng sử dụng Kubernetes mà không cần phải tự cài đặt và quản lý nó.
    """,
    """
    Public Cluster và Private Cluster là hai loại cụm Kubernetes chính được hỗ trợ bởi VKS. Mỗi loại có ưu nhược điểm riêng, phù hợp với những nhu cầu bảo mật và sử dụng khác nhau.

    Dưới đây là bảng so sánh chi tiết giữa Public Cluster và Private Cluster trong VKS:

    Tính năng	Public Cluster	Private Cluster
    Kết nối	Sử dụng địa chỉ Public IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud	Sử dụng địa chỉ Private IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud
    Bảo mật	Bảo mật trung bình do các kết nối sử dụng Public IP	Bảo mật cao hơn với tất cả kết nối đều private và giới hạn truy cập
    Quản lý truy cập	Khó kiểm soát hơn, có thể quản lý truy cập thông qua tính năng Whitelist	Kiểm soát truy cập chặt chẽ, mọi kết nối đều nằm trong mạng private của VNG Cloud
    Khả năng mở rộng (AutoScaling)	Dễ dàng mở rộng thông qua tính năng Auto Scaling	Dễ dàng mở rộng thông qua tính năng Auto Scaling
    Khả năng tự hồi phục (AutoHealing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)
    Khả năng truy cập từ bên ngoài	Dễ dàng truy cập từ bất kỳ đâu	Truy cập từ bên ngoài phải qua các giải pháp bảo mật khác
    Cấu hình và triển khai	Đơn giản hơn do không yêu cầu thiết lập mạng nội bộ	Phức tạp hơn, yêu cầu cấu hình mạng private và bảo mật
    Chi phí	Thường thấp hơn do không cần thiết lập cơ sở hạ tầng bảo mật phức tạp	Chi phí cao hơn do yêu cầu thêm các thành phần bảo mật và quản lý
    Tính linh hoạt	Cao, dễ dàng thay đổi và truy cập các dịch vụ	Linh hoạt hơn trong các ứng dụng yêu cầu bảo mật, nhưng ít linh hoạt hơn cho các ứng dụng yêu cầu truy cập từ bên ngoài
    Phù hợp với	Các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm	Các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư
    Kết luận:

    Public Cluster phù hợp cho các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm. Dễ dàng triển khai và quản lý nhưng có rủi ro bảo mật cao hơn.
    Private Cluster phù hợp cho các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư. Mang lại kết nối ổn định và bảo mật, nhưng yêu cầu cấu hình và quản lý phức tạp hơn, cũng như chi phí cao hơn.
    Bạn cần cân nhắc kỹ lưỡng nhu cầu của mình để lựa chọn loại cluster phù hợp nhất.
    """
]

answers = []
contexts = []

for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# Convert dict to dataset
dataset = EvaluationDataset(
    samples=[
        SingleTurnSample(
            user_input=questions[0],
            retrieved_contexts=contexts[0],
            response=answers[0],
            reference=ground_truths[0]
        ),
        SingleTurnSample(
            user_input=questions[1],
            retrieved_contexts=contexts[1],
            response=answers[1],
            reference=ground_truths[1]
        ),
        SingleTurnSample(
            user_input=questions[2],
            retrieved_contexts=contexts[2],
            response=answers[2],
            reference=ground_truths[2]
        )
    ]
)


## Evaluating the RAG application


In [92]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


In [94]:
result = evaluate(
    llm=llm,
    embeddings=embeddings_model,
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-flash. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-flash. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds 

In [96]:
result

{'context_precision': 1.0000, 'context_recall': 0.7786, 'faithfulness': 1.0000, 'answer_relevancy': 0.8655}

In [98]:
df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS cung cấp nhiều tính năng nổi bật như quản ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.0,0.785714,1.0,0.917253
1,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS là viết tắt của VNGCloud Kubernetes Servic...,\n VKS là viết tắt của VNGCloud Kubernetes ...,1.0,0.800000,1.0,0.945761
2,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,Public Cluster sử dụng địa chỉ IP Public để gi...,\n Public Cluster và Private Cluster là hai...,1.0,0.750000,1.0,0.733616
